# Notebook for run model

너를 위한 것이란다

## Useful functions

밑에있는 함수들을 사용하렴.. print 대신 print_log를 사용하면 중간에 이 노트북을 꺼도 된단다

In [1]:
def print_log(*args, **kwargs) :
    with open('running_log.log', 'a') as f :
        print(*args, file=f, **kwargs)
    print(*args, **kwargs)

아직 정해진게 아무것도 없다...

### some ideas
1) encoding
- image의 형태는 변하지 않고 위치만 변한다
- 따라서 base image를 추출하면 편함
- Red number는 항상 위에 있으므로 추출 편함
- Green number는 가장 green component가 많은 image에서 추출
- Red number는 좌표를 얻기 편하다
- Green number는 가장 일치하는 점이 많은 곳에서 좌표
- Green number가 어렵다 흠..

2) RNN
- 좌표의 sequence 처리

3) decoding
- 좌표 + base image